In [50]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


56040

In [51]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules', '/tmp/tmpogab98cr']


In [71]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [72]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [78]:
from adapters import AutoAdapterModel
from tqdm import tqdm

from transformers import AutoConfig, DistilBertForSequenceClassification

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [79]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack
#lang_adapter_config = AdapterConfig.load("lora", reduction_factor=2)
#lang_adapter_config = AdapterConfig.load("seq_bn", reduction_factor=2)

#lora_mlm_target = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/lora-mlm-target",with_head=False,config="lora")
#prefix_tuning_for_task = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/prefix_tuning_for_task",config="prefix_tuning")
try:
    model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/source-mlm",with_head=False)
    model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/standalone-mlm-target", with_head=False)
except FileNotFoundError as e:
    print(f"Error loading adapter: {e}")

#model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/task_adapter")



In [80]:
import adapters.composition as ac

#model.add_adapter_fusion(ac.Fuse(prefix_tuning_for_task, lora_mlm_target))
#adapter_setup_fuse = ac.Fuse( prefix_tuning_for_task, lora_mlm_target)
#model.train_adapter_fusion(adapter_setup_fuse)

adapter_name= "adapter_test"
lang_adapter_config = AdapterConfig.load("prefix_tuning",leave_out=[0])
model.add_adapter("mnli",config=lang_adapter_config)


model.add_classification_head("mnli", num_labels=3)

model.train_adapter(["mnli"])

model.active_adapters = ac.Stack("mnli","source-mlm")
fn.print_trainable_parameters(model)


trainable params: 5572157 || all params: 79026749 || trainable%: 7.050975866412017


In [83]:
!pip install torchsummary


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [85]:
# For TensorBoard visualization
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
writer.add_graph(model, source_data_eval[0]['input_ids'])  # Here, inputs must include whatever the specific model requires
writer.close()


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/adapters/composition.py:243: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if tensor is not None and hidden_states.shape[0] >= tensor.shape[0]:
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/adapters/composition.py:254: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  max_bsz = max(query.shape[0], key.shape[0], value.shape[0])
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/

ValueError: Invalid adapter setup: str is not a valid adapter name or composition block.

In [76]:
model.adapter_summary(as_dict=True)

[{'name': 'source-mlm',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': False,
  '%param': 5.343131581992825},
 {'name': 'standalone-mlm-target',
  'architecture': 'bottleneck',
  'active': False,
  '#param': 3545856,
  'train': False,
  '%param': 5.343131581992825},
 {'name': 'mnli',
  'architecture': 'prefix_tuning',
  'active': True,
  '#param': 4356608,
  'train': True,
  '%param': 6.564826601859353},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [77]:
try:
    # Evaluate the model on the target domain before adaptation
    reload(fn)
    accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
    print(f"Accuracy before adaptation: {accuracy_before}")
    print(f"F1 score before adaptation: {f1_before}")
except Exception as e:
        print("Error setting up adapter:", str(e))

Error setting up adapter: Invalid adapter setup: str is not a valid adapter name or composition block.


In [26]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.mnli.self.wte.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.mnli.self.control_trans.0.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.mnli.self.control_trans.0.bias
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.mnli.self.control_trans.2.weight
distilbert.transformer.layer.0.attention.prefix_tuning.pool.prefix_tunings.mnli.self.control_trans.2.bias
heads.default.0.weight
heads.default.0.bias
heads.default.2.weight
heads.default.2.bias
heads.mnli.1.weight
heads.mnli.1.bias
heads.mnli.4.weight
heads.mnli.4.bias


In [27]:
reload(config)
reload(fn)
trainer = fn.train_model(model,"lora-for-task",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

ValueError: Invalid adapter setup: str is not a valid adapter name or composition block.

In [106]:
trainer.evaluate(test_target_data)


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.7371801733970642,
 'eval_accuracy': 0.6811740890688259,
 'eval_f1': 0.6816810492478558,
 'eval_precision': 0.6931641833533766,
 'eval_recall': 0.6811740890688259,
 'eval_runtime': 2.836,
 'eval_samples_per_second': 696.763,
 'eval_steps_per_second': 21.862,
 'epoch': 4.0}

In [107]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.6811740890688259
F1 score before adaptation: 0.6816810492478558


In [28]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name,with_head=True)